In [2]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
# using Interact # Make video in notebook
using Reel # Save video as gif
using CSV # For writing to csv
using DataFrames # For writing to csv
using Distributions
using Test
using StatsBase
using Dates
using Plots
using JLD
using DelimitedFiles # To write to text file
using LinearAlgebra # For norm
using Random # For seed to enable repeatability
using NPZ # For providing numpy readable data to RansML
using FileIO # For sampling vehicles that live together in sample_multiple_trajdata
using GaussianMixtures
using PGFPlots
using TikzPictures
using ImageMagick


In [10]:
include("admin_functions.jl")

to_particleMatrix

### Functions

In [250]:
function get_scene(framenum::Int64,traj)
    """ Get a specific scene from traj_ngsim """
    
    scene = Scene(500)
    get!(scene,traj,framenum)
    return scene
end


function get_veh_info(scene;
        car_id = -1,traj=traj_interaction)
    """ Get position and velocity of specific vehicle from scene """ 
 
    @assert car_id>0
    pos=scene.entities[findfirst(car_id,scene)].state.posF.s
    vel = scene.entities[findfirst(car_id,scene)].state.v
    return pos,vel
end


function keep_vehicle_subset!(scene::Scene, ids::Vector{Int})
    """ Remove those vehicles from video who we do not learn filter for.
    Obtained from `ngsim_env/julia/src/ngsim_utils.jl`

    # Example
    ```julia
    scene = Scene(500)
    get!(scene,traj_ngsim,300)
    keep_vehicle_subset!(scene,veh_id_list)
    ```
    """
    
    keep_ids = Set(ids)
    scene_ids = Set([veh.id for veh in scene])
    remove_ids = setdiff(scene_ids, keep_ids)
    for id in remove_ids
        delete!(scene, id)
    end
    return scene
end

keep_vehicle_subset! (generic function with 1 method)

In [12]:

function plot_particles(p_set_mat::Array{Float64,2},time::Float64)
    """ Plot the distribution of particles
    # Caution:
        Only work for 2 parameters. Not developed for 3 parameter case yet

    # Example:
    ```julia
    set_for_plotting = to_particleMatrix(p_set_new)
    plots = []
    push!(plots,plot_particles(set_for_plotting,framenum*0.04))
    ```
    """
    # Check that number of params does not exceed 3

    @assert size(p_set_mat,1) <= 3
    plt = Plots.plot()
    # 2 parameter case	
    if size(p_set_mat,1) == 2
        plt = scatter!(p_set_mat[1,:],p_set_mat[2,:],
            leg=false,title="time=$(time)",xlim=(0,20),ylim=(0,4),
                xlabel="v_des(m/s)",ylabel="sigma")

    # 3 parameter case
    else
        plt = scatter!(p_set_mat[1,:],p_set_mat[2,:],p_set_mat[3,:],leg=false)
        scatter!([true_params[1]],[true_params[2]],[true_params[3]])
    end
    return plt
end


function avg_dist_particles(p_mat,p_fin)
    """ Compute the avg distance over particle set from final particle

    # Arguments
    - `p_mat`: Matrix with particles in every column
    - `p_fin`: 2x1 array with the final particle
    """

    return sum(sqrt.(sum((p_mat .- p_fin).^2,dims=1)))*1/size(p_mat,2)
end


avg_dist_particles (generic function with 1 method)

In [342]:
""" Overlaying hallucinated trajectory on the ground truth

# Fields
- `color::Colorant`
- `scene::Scene`
"""
struct my_overlay <: SceneOverlay
    scene::Scene
    color # Needs to be of form colorant"Colorname"
end

function AutoViz.render!(rendermodel::RenderModel,overlay::my_overlay, 
        scene::Scene, roadway::Roadway)
    AutoViz.render!(rendermodel, overlay.scene, car_color = overlay.color)
    return rendermodel
end


""" Display the ID on top of each entity in a scene.

# Fields
- `color::Colorant`
- `font_size::Int64`
"""
mutable struct IDOverlay <: SceneOverlay
    color::Colorant
    font_size::Int
end

function AutoViz.render!(rendermodel::RenderModel, overlay::IDOverlay, scene::Scene, 
                            env::E) where E
    font_size = overlay.font_size
    for veh in scene
        add_instruction!(rendermodel, render_text, ("$(veh.id)", veh.state.posG.x, 
                        veh.state.posG.y, font_size, overlay.color), incameraframe=true)
    end
    return rendermodel
end


"""
    curve_pts_overlay
Displays circles at the curve points that constitute the lanes of a road: 
- color: the color of the point
- size: the size of the point

# Examples
```julia
render(scene,road,[curvepts_overlay(roadway_ext,colorant"yellow",0.05)])
```
""" 
struct curvepts_overlay <: SceneOverlay
    roadway
    color::Colorant # eg: colorant"yellow"
    size::Float64
end

function AutoViz.render!(rendermodel::RenderModel, overlay::curvepts_overlay, 
        scene::Frame{Entity{S,D,I}}, roadway::R) where {S,D,I,R}
    
    num_segments = length(roadway.segments)
    
    # Loop over segments
    for seg_i in 1:num_segments
        seg = roadway.segments[seg_i]
        num_lanes = length(seg.lanes)
        
        # Within a segment, loop over lanes
        for lane_i in 1:num_lanes
            lane = seg.lanes[lane_i]
            
            # Within a lane, loop over the curve points
            num_curvepts = length(lane.curve)
            for cpt_i in 1:num_curvepts
                cpt = lane.curve[cpt_i]
                
                add_instruction!(rendermodel,render_circle,
                    (cpt.pos.x,cpt.pos.y,overlay.size,overlay.color))
            end
        end
    end
    
end


In [411]:
function make_gif(plots;filename="output.mp4")
    """ Make a video using an array of plots. Uses the Reel library """
    @assert typeof(filename) == String
    #@show "Making gif"
    frames = Frames(MIME("image/png"), fps=10)
    for plt in plots
        push!(frames, plt)
    end
    write(string("media/",filename), frames)
    return nothing
end # End of the reel gif writing function



# Make demonstration traj video but highlighting vehicles in veh_id_list
"""
    function make_video

- Make a video highlighting a selection of vehicles from replay demonstration trajectory
- Overlay vehicle ids for vehicles in 

# Overlays
- Id number of all vehicles
- Timestep as a text overlay

# Example
```julia
# random sample vehicle selection with seed 1 for 10 vehicles 50 driving steps
id_list = [1101, 1209, 1126, 1200, 1171, 1119, 1131, 1127, 1154, 1202]
make_video(range=3273:3323,veh_id_list=id_list)
```
"""
function make_video(;range=nothing, veh_id_list=[],
        traj=traj_ngsim, roadway=roadway_ngsim, plot_curvepts=false, C=nothing, zoom=6.,
        filename="media/ngsim_vids/replay_$(range).mp4")

    frames = Frames(MIME("image/png"), fps=10)
    scene = Scene(500)
    
    if C == "ngsim"
        C = VecE2(0.0,0.0)
        n_veh = 0
        for i in range
            scene_i = get_scene(i,traj)
            if !isempty(veh_id_list) keep_vehicle_subset!(scene_i,veh_id_list) end 
            for veh in scene_i
                C += convert(VecE2, veh.state.posG)
            end
            n_veh += length(scene_i)
        end
        C = C / n_veh
    end

    if C == "interaction"
        roadway_X = [cpt_k.pos.x for seg in roadway.segments for lane in seg.lanes for cpt_k in lane.curve]
        roadway_Y = [cpt_k.pos.y for seg in roadway.segments for lane in seg.lanes for cpt_k in lane.curve]

        C_X = (minimum(roadway_X) + maximum(roadway_X)) / 2
        C_Y = (minimum(roadway_Y) + maximum(roadway_Y)) / 2
        C = VecE2(C_X, C_Y)
    end
    print("center: $C\n")
    
    
    for i in range
        temp_scene = get_scene(i,traj)
        
        # Color vehicles in id_list green and show their vehicle ids
        if !isempty(veh_id_list)
            carcolors = Dict{Int,Colorant}()
            for veh in temp_scene
                if veh.id in veh_id_list
                    carcolors[veh.id] = color_target
                else
                    carcolors[veh.id] = color_background
                end
            end
        end
        
        if plot_curvepts == true
            Overlays = [curvepts_overlay(roadway,colorant"yellow",0.02), 
                IDOverlay(colorant"white", 12), TextOverlay(text=["t=$(i)"],font_size=14)]
        else
            Overlays = [IDOverlay(colorant"white", 12), TextOverlay(text=["t=$(i)"],font_size=14)]
        end
        
        scene_visual = render(temp_scene, roadway, 
                Overlays, car_colors = carcolors,
                cam = StaticCamera(C, zoom))
        push!(frames,scene_visual)
    end
    write(filename,frames)
    print("Making video called $filename")
    
    return nothing
end

make_video

In [304]:
veh_id_list = [134,135,136,141,144,142,147,137,139,152,151,153,155,156,157,162,164]
make_video(range=700:800, veh_id_list=veh_id_list, 
    traj=traj_interaction, roadway=roadway_interaction_extension, plot_curvepts=true, C=C, zoom=6.,
    filename="media/interaction_vids/replay_interaction_700-800.mp4")

center: VecE2(1073.574, 966.638)
Making video called media/interaction_vids/replay_interaction_700-800.mp4

In [100]:
function sample_init_particles(num_p::Int64;v::Float64=30.0)
"""
    function sample_init_particles(num_p::Int64;v=30.0)

Inspired from `gen_test_particles` in `admin.jl`.
Samples `num_p` particles neighbourhood around the input `v`

# Used by
- `multistep_update`

# Example
```julia
sample_init_particles(10,v=20.0)
```
"""
    v_particles = sample(v-2.0:0.5:v+2.0,num_p)
    sig_particles = sample(1.0:0.1:5.0,num_p)
    p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)
    return p_set_dict
end




function addnoise(p_set_mat,weight_vec)
"""
    function addnoisepar
Add noise to elite particles to counter the particle deprivation problem
- Used by `multistep_update`
# Arguments
- `p_set_mat`: Matrix with each column being a particle. Row1 has `v_des` and row2 has `sigma`
- `weight_vec`: Associated weight for particles. To capture the elite 20% for noise addition 

# Returns
- `new_set` Set of particles with noise added

# Example
See `multistep_update`
```julia
num_p = 10
v = 30.
v_particles = rand(v-5.0:0.5:v+5.0,1,num_p)
sig_particles = rand(0.1:0.1:3.0,1,num_p)
p_set = [v_particles;sig_particles]
@show p_set
scatter(p_set[1,:],p_set[2,:],xlim=(22,36),ylim=(0,4),leg=false,markercolor = "red")
new_p = addnoise(p_set)
@show new_p
scatter!(new_p[1,:],new_p[2,:],xlim=(22,36),ylim=(0,4),leg=false,markercolor="green")
```
"""
    # Select the elite particles
    num_p = size(p_set_mat,2)
    num_top = Int(ceil(0.2*num_p))
    
    sorted_idx = sortperm(weight_vec,rev=true)[1:num_top] 
    
    # Generate permuter for v_des and sigma
    v_noise = rand([-0.5,0.,0.5],1,num_top)
    sig_noise = rand([-0.1,0.,0.1],1,num_top)
    stacked_noise = [v_noise;sig_noise]
    
    # Add noise only to the elite particles i.e top 20% particles
    p_set_mat[:,sorted_idx] = p_set_mat[:,sorted_idx] + stacked_noise
    new_set = p_set_mat
    
    # Impose bounds to avoid sigma becoming 0
    new_set[2,findall(new_set[2,:] .< 0.1)] .= 0.1
    return new_set
end

addnoise (generic function with 1 method)

In [16]:

function hallucinate_a_step(roadway,scene_input,particle,timestep;car_id=-1)
    """
        function hallucinate_a_step(roadway,scene_input,particle;car_id=-1)
    - Hallucinate `car_id` a step using `particle` starting from `scene` as the true scene
    - Caution: Used `timestep_ngsim` as a global variable

    # Example
    ```julia
    scene=get_scene(300,traj_ngsim)
    particle = Dict(:v_des=>25.0,:σ=>0.5)
    hallucinate_a_step(roadway,scene,particle,car_id=scene.entities[2].id)
    ```
    """
    
    if car_id==-1 @show "Please give valid car_id" end

    scene = deepcopy(scene_input)
    #scene = scene_input # This was the failure case
    n_cars = scene.n 

    models = Dict{Int, DriverModel}()
    
    # Create driver models for all the cars in the scene
    for veh in scene
        if veh.id == car_id
            models[veh.id] = IntelligentDriverModel(;particle...,T=0.2,s_min=0.5)
        else
            # TODO: RESEARCH QUESTION: What drives the other vehicles in the hallucination
            # Chose a high v_des such that the leader does not cause ego to choose a slow vel particle
            models[veh.id] = IntelligentDriverModel(v_des=50.0)
        end
    end

    actions = Array{Any}(undef,length(scene))
    get_actions!(actions,scene,roadway,models)
    tick!(scene,roadway,actions,timestep)
    
    halluc_pos = scene.entities[findfirst(car_id,scene)].state.posF.s
    return halluc_pos
end


function compute_particle_likelihoods(roadway,f,trupos,p_set_dict,timestep; car_id=-1)
    """
    - compute likelihood
    
    # Caution
    - `timestep` defined in the function. Was 0.04 for highd. Here is 0.1
    """
    
    if car_id==-1 @show "Please give valid car_id" end
    timestep = 0.1 #TODO: Remove hardcoding
    p_mat, params, vec_val_vec = to_matrix_form(p_set_dict)
    
    num_params=size(p_mat)[1]
    num_p = size(p_mat)[2]
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p    
        # Create dict version for a single particle
        p_dict = Dict()
        for j in 1:num_params
            p_dict[params[j]]=vec_val_vec[j][i]
        end
        
        std_dev_acc = p_dict[:σ]
        
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc <= 0 std_dev_acc = 0.1 end
        
        # TODO: This math needs to be verified from random variable calculations
        std_dev_pos = timestep*timestep*std_dev_acc

        hpos = hallucinate_a_step(roadway,f,p_dict,timestep;car_id=car_id)
        lkhd_vec[i] = pdf(Normal(hpos,std_dev_pos),trupos[1])
    end
    return lkhd_vec,p_mat,params
end


function update_p_one_step(f,trupos,p_set_dict,timestep;
        car_id=-1,roadway=roadway_interaction)
    """
        function update_p_one_step(roadway,f,trupos,p_set_dict;car_id=-1)
    - Update particles a step, New particle set based on one step of resampling
    - Erroneous weights calculation. Had negative weights at some point

    # Returns
    - `new_p_set_dict`: Dictionary with keys as parameters and values as array of particles
    - `p_weight_vec`: Vector with weights associated to the particles in `new_p_set_dict`
    """
    
    if car_id==-1 @show "Provide valid car_id" end

    lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,f,trupos,p_set_dict,timestep,
                                                        car_id=car_id)

    num_params = size(p_mat)[1]
    num_p = size(p_mat)[2]
    
    p_weight_vec = StatsBase.weights(lkhd_vec./sum(lkhd_vec)) # Convert to weights form to use julia sampling
    idx = sample(1:num_p,p_weight_vec,num_p)
    new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
    
    new_p_set_dict = to_dict_form(params,new_p_mat)
    return new_p_set_dict, p_weight_vec
end


update_p_one_step (generic function with 1 method)

In [449]:
# Functions: multiple steps of filtering, obtain driver models for multiple cars
"""
    function multistep_update(num_p::Int64,car_id::Int64,
        start_frame::Int64,last_frame::Int64;p_dist_vid=false)

Returns the mean particle after running multiple steps of filtering

# Arguments
- `start_frame`: Frame to start filtering from
- `last_frame`: Frame to end filtering at
- `num_p`: Number of particles

# Keyworded arguments
- `p_dist_vid`: Default false. Makes a video of particle distribution if true

# Returns
- `mean_particle`: Dict containing the mean values of the final particle set after filtering
- `iterwise_p_set`: List with particle set (in the matrix form) at each iteration

# Example
```julia
# Make a video for particle evolution for vehicle 13 starting from
# frame 11 and ending at frame number 60
multistep_update(500,13,11,60,p_dist_vid=true)
```
"""
function multistep_update(num_p::Int64, car_id::Int64,
        start_frame::Int64, last_frame::Int64, timestep;
        p_dist_vid=false, traj=traj_interaction, roadway=roadway_interaction_extension)

    startscene = get_scene(start_frame,traj)
    startpos,startvel = get_veh_info(startscene,car_id=car_id, traj=traj)

    p_set_dict = sample_init_particles(num_p,v=startvel)
    
    plots = []
    init_p_mat = to_particleMatrix(p_set_dict)
    push!(plots,plot_particles(init_p_mat,0.0))
    
    dt = 0.1 # Multiplies framenum in creating the plot

    iterwise_p_set = [] # Stores particle set at every iteration
    push!(iterwise_p_set,init_p_mat)
    
    for framenum in start_frame:last_frame
#         print("multistep says. Framenum = $(framenum) \n")
        scene = get_scene(framenum+1,traj)
        
        if car_id in [veh.id for veh in scene]
            trupos,truvel = get_veh_info(scene,car_id=car_id, traj=traj)

            scene = get_scene(framenum,traj)
            p_set_new, weight_vec = update_p_one_step(scene,trupos,p_set_dict,timestep; car_id=car_id, roadway=roadway)

            params = [:v_des,:σ]
            p_set_dict = to_dict_form(params,addnoise(to_particleMatrix(p_set_new),weight_vec))
            set_for_plotting = to_particleMatrix(p_set_dict)
            push!(plots,plot_particles(set_for_plotting,framenum*dt))
            push!(iterwise_p_set,set_for_plotting)
            
        end
    end
    mean_particle = find_mean_particle(p_set_dict)

#         # Make a plot of the particle distribution evolution
#     if p_dist_vid 
#         print("Making a particle distribution video\n")
#         make_gif(plots,filename = "vehid_$(car_id).gif") 
# #         date = Dates.now()
# #         make_gif(plots,filename = "vehid_$(car_id)_$(date).gif") 
#     end
    
    return mean_particle,iterwise_p_set
end


"""
    function obtain_driver_models(veh_id_list,num_particles,start_frame,last_frame)
Driver models for each vehicle in veh_id_list

# Arguments
- `veh_id_list` List with vehicle ids
- `start_frame` Frame to start filtering from
- `last_frame` Frame to end hallucination at

# Returns
- `models` Dict with veh id as key and IDM driver model as value
- `final_particles` Dict with veh id as key and avg particle over final particle set as value
- `mean_dist_mat`: Every elem is the mean dist of particle set at that iter (row) for that car (column)

# Example
```julia
veh_id_list = [72,75,73,67,69,71,64,59,56,57,62,60,54,55,49,51,48,43,47,39,37,34,44,33,31]
model_multimodels,particles_multimodels,mean_dist_mat = obtain_driver_models(veh_id_list,500,300,375)

# Example 2
veh_id_list = [756,758,759,761,762,763,765,767,771,773,775,776,778,779,782,784,785]
new_models, = obtain_driver_models(veh_id_list,500,2000,2090)
```
"""
function obtain_driver_models(veh_id_list,num_particles,start_frame,last_frame; 
        timestep=timestep_interaction, traj=traj_interaction, roadway=roadway_interaction_extension)
    
    models = Dict{Int64,DriverModel}() # key is vehicle id, value is driver model
    final_particles = Dict{Int64,Dict}() # key is vehicle id, value is final particles
    
        # Loop over all the cars and get their corresponding IDM parameters by particle filter
    num_cars = length(veh_id_list)
    num_iters = last_frame-start_frame+2 # NEED TO CONFIRM THIS
    
        # num_iters x num_cars. Every elem is the mean dist of particle set at that iter for that car
    mean_dist_mat = fill(0.,num_iters,num_cars)
    
    for (ii,veh_id) in enumerate(veh_id_list)
        print("obtain_driver_models. vehicle id = $(veh_id) \n")
        
        mean_particle, iterwise_p_set = multistep_update(num_particles,veh_id,start_frame,last_frame,timestep; 
            p_dist_vid=true, traj=traj, roadway=roadway)
        print("mean_particle", mean_particle, "\n")
        
        final_particles[veh_id] = mean_particle
        models[veh_id] = IntelligentDriverModel(;mean_particle...) #,T=0.2,s_min=1.)
        
            # Get the mean distance from final particle over iterations
        p_fin = fill(0.,2,1)
        i = 0
        for (k,v) in mean_particle
            i = i+1
            p_fin[i] = v
        end
        
        # num_iters = length(iterwise_p_set) # SHOULD MATCH OUTSIDE LOOP VARIABLE
        mean_dist_over_iters = fill(0.,num_iters,1)
        for (jj,p_mat) in enumerate(iterwise_p_set)
            mean_dist_over_iters[jj,1] = avg_dist_particles(p_mat,p_fin)
        end
        
        mean_dist_mat[:,ii] = mean_dist_over_iters    
    end
    
        # Average over the cars and plot the filtering progress over frames
    #avg_over_cars = mean(mean_dist_mat,dims=2)
    #plot(avg_over_cars)
    
    return models, final_particles, mean_dist_mat
end

obtain_driver_models

In [408]:

"""
    function run_simulation_extract_rmse(;start_step,nsteps,roadway,timestep,id_list,models)
- Drive vehicles in `id_list` starting from frame number `start_step` for `nsteps`
- Driver models are provided in `models`
- Extract rmse postion and velocity against ground truth

# Arguments
- `nsteps`: Number of steps to do the simulation
- `models`: The driver models to drive the simulation
- `roadway`: The roadway
- `timestep`: Timestep (0.1 for ngsim, 0.04 for highd)
- `id_list`: List of ids of vehicles of interest

# Other functions used
`get_scene`,`AutomotiveDrivingModels.get_actions!`,`AutomotiveDrivingModels.tick!`,`my_overlay`

# Returns
- `rmse_pos::Dict{Int64,Vector{Float64}}`:  Key is vehid. Value is array. Each elem is timewise rmse pos value for that veh_id
- `rmse_vel::Dict{Int64,Vector{Float64}}`: Key is vehid. Value is array. Each elem is timewise rmse vel value for that veh_id
- `out.mp4`: Video of hallucination overlayed with ground truth

# Examples
```julia
rmse_pos_dict, = run_simulation_extraxct_rmse(start_step = 300,nsteps=100,
    timestep=timestep_ngsim,roadway=roadway_ngsim,models=best_models,id_list=veh_id_list);
```
"""
function run_simulation_extract_rmse(;start_step, nsteps, id_list, models,
        timestep=timestep_ngsim, traj=traj_ngsim, roadway=roadway_ngsim,
        filename = "media/ngsim_vids/rmse_ngsim.mp4", makevideo=true, 
        plot_curvepts=false, C=nothing, zoom=6.)    
        
        # Center point for visulization
    if C == "ngsim"
        C = VecE2(0.0,0.0)
        n_veh = 0
        for (i,t) in enumerate(start_step:start_step+nsteps-1)
            scene_t = get_scene(t,traj)
            if !isempty(veh_id_list) keep_vehicle_subset!(scene_t,veh_id_list) end 
            for veh in scene_t
                C += convert(VecE2, veh.state.posG)
            end
            n_veh += length(scene_t)
        end
        C = C / n_veh
    end

    if C == "interaction"
        roadway_X = [cpt_k.pos.x for seg in roadway.segments for lane in seg.lanes for cpt_k in lane.curve]
        roadway_Y = [cpt_k.pos.y for seg in roadway.segments for lane in seg.lanes for cpt_k in lane.curve]

        C_X = (minimum(roadway_X) + maximum(roadway_X)) / 2
        C_Y = (minimum(roadway_Y) + maximum(roadway_Y)) / 2
        C = VecE2(C_X, C_Y)
    end
    print("center: $C\n")
    
    
        # Setting up
    rmse_pos = Dict{Int64,Vector{Float64}}()
    rmse_vel = Dict{Int64,Vector{Float64}}()
    
    for veh_id in id_list
        rmse_pos[veh_id] = []
        rmse_vel[veh_id] = []
    end
    
    frames = Frames(MIME("image/png"), fps=7) # For making video
    scene_halluc = get_scene(start_step,traj) # Frame to start hallucination from
    
    for (i,t) in enumerate(start_step:start_step+nsteps-1)
#         print("run_simulation_extract_rmse, step $t \n")
        if !isempty(id_list) keep_vehicle_subset!(scene_halluc,id_list) end

        actions = Array{Any}(undef,length(scene_halluc))
        get_actions!(actions,scene_halluc,roadway,models)
        tick!(scene_halluc,roadway,actions,timestep)

        scene_target = get_scene(t+1,traj)
#         if !isempty(id_list) keep_vehicle_subset!(scene_target,id_list) end
        
            # rmse extraction
        for veh_id in id_list
            if veh_id in [veh.id for veh in scene_target]
                target_veh = scene_target[findfirst(veh_id,scene_target)]
                ego_veh = scene_halluc[findfirst(veh_id,scene_halluc)]

                push!(rmse_pos[veh_id], norm(target_veh.state.posG[1:2]-ego_veh.state.posG[1:2]))
                push!(rmse_vel[veh_id], norm(target_veh.state.v - ego_veh.state.v))
            end
        end
        
        
        if !isempty(id_list)
            carcolors = Dict{Int,Colorant}()
            for veh in scene_target
                if veh.id in veh_id_list
                    carcolors[veh.id] = color_target
                else
                    carcolors[veh.id] = color_background
                end
            end     
#             veh_list = [scene_halluc[findfirst(veh_id,scene_halluc)] for veh_id in id_list]
#             HallucOverlay = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
#                     pos=VecE2(veh.state.posG.x-0.7, veh.state.posG.y-0.1)) for veh in veh_list]
        end   
        
        HallucOverlay = my_overlay(scene_halluc, color_halluc)
        if plot_curvepts == true
            Overlays = [curvepts_overlay(roadway,colorant"yellow",0.02), HallucOverlay, 
                IDOverlay(colorant"white", 12), TextOverlay(text=["t=$(t)"],font_size=14)]
        else
            Overlays = [HallucOverlay,
                IDOverlay(colorant"white", 12), TextOverlay(text=["t=$(t)"],font_size=14)]
        end
           
        scene_visual = render(scene_target, roadway, 
                Overlays, car_colors = carcolors, cam=StaticCamera(C, zoom))
        push!(frames,scene_visual)
    end
    
    if makevideo
        print("Making video filename: $(filename)\n")
        write(filename,frames)
    end

    return rmse_pos,rmse_vel
end


run_simulation_extract_rmse

## Interaction dataset

In [4]:
include("../src/veh_track_reading.jl");
include("../src/roadway_building.jl");

timestep_interaction=0.1;

In [388]:
roadway_interaction = make_roadway_interaction()
roadway_interaction_extension = make_roadway_interaction_with_extensions();

In [8]:
traj_interaction = read_veh_tracks(roadway=roadway_interaction);

In [426]:
roadway_X = [cpt_k.pos.x for seg in roadway_interaction.segments for lane in seg.lanes for cpt_k in lane.curve]
roadway_Y = [cpt_k.pos.y for seg in roadway_interaction.segments for lane in seg.lanes for cpt_k in lane.curve]

C_X = (minimum(roadway_X) + maximum(roadway_X)) / 2
C_Y = (minimum(roadway_Y) + maximum(roadway_Y)) / 2
C = VecE2(C_X, C_Y)
print("center: $C\n")
    

center: VecE2(1070.962, 955.718)


In [367]:
color_target = colorant"#004d99"  #pastel-blue
# color_target = colorant"#0090bc"
color_background = colorant"#b3d9ff"  #pastel-skyblue
color_halluc = colorant"#ffa64d";


In [326]:
veh_id_list = [134,135,136,141,144,142,147,137,139,152,151,153,155,156] #,157,162,164]

make_video(range=700:750, veh_id_list=veh_id_list, 
    traj=traj_interaction, roadway=roadway_interaction_extension, 
    plot_curvepts=false, C=C, zoom=6.5,
    filename="media/interaction_vids/replay_interaction_700-750.mp4")

Making video called media/interaction_vids/replay_interaction_700-800.mp4

In [448]:
veh_id_list = [134,135,136,141,144,142,147,137,139,152,151,153,155,156,157] #,162,164]
# veh_id_list = [137,139,144,142,147,152,151,153,155,156,157]

model_multimodels,particles_multimodels,mean_dist_mat = obtain_driver_models(veh_id_list, 
    500,700,780; traj=traj_interaction, roadway=roadway_interaction_extension)

init_pos_dict, init_vel_dict= run_simulation_extract_rmse(start_step=700,nsteps=80, id_list=veh_id_list,
    timestep=timestep_interaction, traj=traj_interaction, roadway=roadway_interaction_extension,
    models=model_multimodels, plot_curvepts=false, C=C, zoom=6.5,
    filename = "media/interaction_vids/rmse_interaction_pfIDM_MOBIL_700-780.mp4")
print("Rmse extracted: pfIDM models\n")


obtain_driver_models. vehicle id = 134 
mean_particleDict{Any,Any}(:v_des=>8.95612,:σ=>5.6684)
obtain_driver_models. vehicle id = 135 
mean_particleDict{Any,Any}(:v_des=>5.09728,:σ=>4.9846)
obtain_driver_models. vehicle id = 136 
mean_particleDict{Any,Any}(:v_des=>5.3862,:σ=>5.2052)
obtain_driver_models. vehicle id = 141 
mean_particleDict{Any,Any}(:v_des=>5.30047,:σ=>4.0332)
obtain_driver_models. vehicle id = 144 
mean_particleDict{Any,Any}(:v_des=>8.28182,:σ=>5.5574)
obtain_driver_models. vehicle id = 142 
mean_particleDict{Any,Any}(:v_des=>6.64013,:σ=>2.5816)
obtain_driver_models. vehicle id = 147 
mean_particleDict{Any,Any}(:v_des=>4.54313,:σ=>2.2626)
obtain_driver_models. vehicle id = 137 
mean_particleDict{Any,Any}(:v_des=>6.39868,:σ=>5.3836)
obtain_driver_models. vehicle id = 139 
mean_particleDict{Any,Any}(:v_des=>4.98759,:σ=>4.887)
obtain_driver_models. vehicle id = 152 
mean_particleDict{Any,Any}(:v_des=>7.14511,:σ=>3.2788)
obtain_driver_models. vehicle id = 151 
mean_particl

### NGSIM dataset

In [382]:
# Loading: ngsim roads and trajectories
traj_ngsim = open(io->read(io, MIME"text/plain"(), Trajdata), 
    "../dataset/trajdata_i101_trajectories-0750am-0805am.txt", "r");
roadway_ngsim = open(io->read(io, MIME"text/plain"(), Roadway), 
    "../dataset/ngsim_101.txt", "r");

timestep_ngsim=0.1;

In [4]:
# Scenario definitions: suffix shows start frame number for a 100 step duration
    # Hold up on left lane
id_list_1000 = [298,302,322,363,376,396,397,404,417,422,427,442,451]
    # No hold ups
id_list_2000 = [756,758,759,761,762,763,765,767,771,773,775,776,778,779,782,784,785]
id_list_300_375 = [72,75,73,67,69,71,64,59,56,57,62,60,54,55,49,51,48,43,47,39,37,34,44,33]; #,31
id_list_3273_3323 = [1101, 1209, 1126, 1200, 1171, 1119, 1131, 1127, 1154, 1202]


In [392]:
roadway_X = [cpt_k.pos.x for seg in roadway_ngsim.segments for lane in seg.lanes for cpt_k in lane.curve]
roadway_Y = [cpt_k.pos.y for seg in roadway_ngsim.segments for lane in seg.lanes for cpt_k in lane.curve]

C_X = (minimum(roadway_X) + maximum(roadway_X)) / 2
C_Y = (minimum(roadway_Y) + maximum(roadway_Y)) / 2
C = VecE2(C_X, C_Y)
print("center: $C\n")


color_target = colorant"#004d99"  #pastel-blue
# color_target = colorant"#0090bc"
color_background = colorant"#b3d9ff"  #pastel-skyblue
color_halluc = colorant"#ffa64d";

center: VecE2(1966559.896, 570767.311)


#### id_list_300_375

In [430]:
veh_id_list = [72,75,73,67,69,71,64,59,56,57,62,60,54,55,49,51,48,43,47,39,37,34,44,42,33,31] 
make_video(range=300:400, veh_id_list=veh_id_list, 
    traj=traj_ngsim, roadway=roadway_ngsim, C="ngsim", zoom=4.,
    filename="media/ngsim_vids/replay_ngsim_300-375.mp4")


center: VecE2(1966508.263, 570801.001)
Making video called media/ngsim_vids/replay_ngsim_300-375.mp4

In [415]:
veh_id_list = [42,33,72,75,73,67,69,71,64,59,56,57,62,60,54,55,49,51,48,43,47,39,37,34]
model_multimodels,particles_multimodels,mean_dist_mat = obtain_driver_models(veh_id_list, 500,300,400; 
    timestep=timestep_ngsim, traj=traj_ngsim, roadway=roadway_ngsim)

init_pos_dict, init_vel_dict= run_simulation_extract_rmse(start_step=300,nsteps=100,id_list=veh_id_list,
    timestep=timestep_ngsim, traj=traj_ngsim, roadway=roadway_ngsim,
    models=model_multimodels, C=C, zoom=4.,
    filename = "media/ngsim_vids/rmse_ngsim_pfIDM_300-375.mp4")
print("Rmse extracted: pfIDM models\n")


obtain_driver_models. vehicle id = 42 
obtain_driver_models. vehicle id = 33 
obtain_driver_models. vehicle id = 72 
obtain_driver_models. vehicle id = 75 
obtain_driver_models. vehicle id = 73 
obtain_driver_models. vehicle id = 67 
obtain_driver_models. vehicle id = 69 
obtain_driver_models. vehicle id = 71 
obtain_driver_models. vehicle id = 64 


BoundsError: BoundsError: attempt to access 300-element UnitRange{Int64} at index [4625404556023066748]

#### id_list_3273_3323

In [413]:
id_list = [1101, 1209, 1126, 1200, 1171, 1119, 1131, 1127, 1154, 1202]
make_video(range=3273:3323, veh_id_list=id_list, traj=traj_ngsim, roadway=roadway_ngsim, C="ngsim", zoom=2.,
    filename="media/ngsim_vids/replay_ngsim_3273-3323.mp4")

center: VecE2(1966537.742, 570776.719)
Making video called media/ngsim_vids/replay_ngsim_3273-3323.mp4

#### id_list_2000_2090

In [402]:
id_list = [756,758,759,761,762,763,765,767,771,773,775,776,778,779,782,784,785]
make_video(range=2000:2090, veh_id_list=id_list, traj=traj_ngsim, roadway=roadway_ngsim, C="ngsim", zoom=4.,
    filename="media/ngsim_vids/replay_ngsim_2000-2090.mp4")


Making video called media/ngsim_vids/replay_ngsim_2000-2090.mp4